[![GitHub Repository](https://img.shields.io/badge/GitHub-Repository-181717?style=for-the-badge&logo=GitHub&link=https://github.com/Mearman/openalex-docs)](https://github.com/Mearman/openalex-docs)[![Open in GitHub](https://img.shields.io/badge/Open%20in-GitHub-181717?style=for-the-badge&logo=github&link=https://github.com/Mearman/openalex-docs/blob/main/api-entities/concepts/filter-concepts.ipynb)](https://github.com/Mearman/openalex-docs/blob/main/api-entities/concepts/filter-concepts.ipynb)[![Open in Colab](https://img.shields.io/badge/Open%20in-Colab-F9AB00?style=for-the-badge&logo=Google%20Colab&link=https://colab.research.google.com/github/Mearman/openalex-docs/blob/main/api-entities/concepts/filter-concepts.ipynb)](https://colab.research.google.com/github/Mearman/openalex-docs/blob/main/api-entities/concepts/filter-concepts.ipynb)

In [ ]:
%pip install --upgrade "git+https://github.com/Mearman/openalex-python-pydantic-v1.git"
%pip install pandasai

In [ ]:
import json
import pandas as pd
import numpy as np
from openalex_api import Configuration, ApiClient, AutocompleteApi, AuthorsApi, ConceptsApi, FundersApi, InstitutionsApi, PublishersApi, SourcesApi, WorksApi

configuration = Configuration(host="https://api.openalex.org")
autocomplete_api = AutocompleteApi(ApiClient(configuration))
authors_api = AuthorsApi(ApiClient(configuration))
concepts_api = ConceptsApi(ApiClient(configuration))
funders_api = FundersApi(ApiClient(configuration))
institutions_api = InstitutionsApi(ApiClient(configuration))
publishers_api = PublishersApi(ApiClient(configuration))
sources_api = SourcesApi(ApiClient(configuration))
works_api = WorksApi(ApiClient(configuration))

from pandasai import SmartDataframe
from pandasai.llm import OpenAI

In [ ]:
# @title  { run: "auto", display-mode: "form" }
openapi_token = "" # @param {type:"string"}

# Filter concepts

You can filter concepts with the `filter` parameter:

* Get concepts that are at level 0 (top level)\
  [`https://api.openalex.org/concepts?filter=level:0`](https://api.openalex.org/concepts?filter=level:0)

In [ ]:
# @title { run: "auto", vertical-output: false }
# https://api.openalex.org/concepts?filter=level:0
filter="level:0" # @param "level:0" {type: "string"}

response = concepts_api.get_concepts(
	filter=filter
)

df = pd.DataFrame(response.results)
display(df)

In [ ]:
numeric_df = df[['id', 'display_name'] +
	[col for col in df.columns if df[col].dtype in ['int64', 'float64'] and col != 'relevance_score']]
display(numeric_df)

try:
	llm = OpenAI(api_token = openapi_token)
	sdf = SmartDataframe(numeric_df, config = { "llm": llm })
	sdf.chat("Plot a chart of this data")
except:
	if not openapi_token:
		print("Error: openapi_token not set")
	else:
		print("Error when creating SmartDataframe")

{% hint style="info" %}
It's best to [read about filters](./../../how-to-use-the-api/get-lists-of-entities/filter-entity-lists.ipynb) before trying these out. It will show you how to combine filters and build an AND, OR, or negation query
{% endhint %}

### `/concepts` attribute filters

You can filter using these attributes of the [`Concept`](./concept-object.ipynb) object (click each one to view their documentation on the `Concept` object page):

* [`ancestors.id`](./concept-object.md#ancestors)
* [`cited_by_count`](./concept-object.md#cited\_by\_count)
* [`ids.openalex`](./concept-object.md#ids) (alias: `openalex`)
* [`level`](./concept-object.md#level)
* [`summary_stats.2yr_mean_citedness`](./concept-object.md#summary_stats) (accepts float, null, !null, can use range queries such as < >)
* [`summary_stats.h_index`](./concept-object.md#summary_stats) (accepts integer, null, !null, can use range queries)
* [`summary_stats.i10_index`](./concept-object.md#summary_stats) (accepts integer, null, !null, can use range queries)
* [`works_count`](./concept-object.md#works\_count)

### `/concepts` convenience filters

These filters aren't attributes of the [`Concept`](./concept-object.ipynb) object, but they're included to address some common use cases:

#### `default.search`

Value: a search string

This works the same as using the [`search` parameter](./search-concepts.md#search-concepts) for Concepts.

#### `display_name.search`

Value: a search string

Returns: concepts with a [`display_name`](./concept-object.md#display\_name) containing the given string; see the [search page](./search-concepts.md#search-a-specific-field) for details.

* Get concepts with `display_name` containing "electrodynamics":\
  [`https://api.openalex.org/concepts?filter=display_name.search:electrodynamics`](https://api.openalex.org/concepts?filter=display\_name.search:electrodynamics)

In [ ]:
# @title { run: "auto", vertical-output: false }
# https://api.openalex.org/concepts?filter=display_name.search:electrodynamics
filter="display_name.search:electrodynamics" # @param "display_name.search:electrodynamics" {type: "string"}

response = concepts_api.get_concepts(
	filter=filter
)

df = pd.DataFrame(response.results)
display(df)

In [ ]:
numeric_df = df[['id', 'display_name'] +
	[col for col in df.columns if df[col].dtype in ['int64', 'float64'] and col != 'relevance_score']]
display(numeric_df)

try:
	llm = OpenAI(api_token = openapi_token)
	sdf = SmartDataframe(numeric_df, config = { "llm": llm })
	sdf.chat("Plot a chart of this data")
except:
	if not openapi_token:
		print("Error: openapi_token not set")
	else:
		print("Error when creating SmartDataframe")

{% hint style="info" %}
In most cases, you should use the [`search` parameter](./search-concepts.md#concepts-full-search) instead of this filter because it uses a better search algorithm.
{% endhint %}

#### `has_wikidata`

Value: a Boolean (`true` or `false`)

Returns: concepts that have or lack a [Wikidata ID](./concept-object.md#wikidata), depending on the given value. For now, all concepts in OpenAlex _do_ have Wikidata IDs.

* Get concepts without Wikidata IDs:\
  [`https://api.openalex.org/concepts?filter=has_wikidata:false`](https://api.openalex.org/concepts?filter=has\_wikidata:false)

In [ ]:
# @title { run: "auto", vertical-output: false }
# https://api.openalex.org/concepts?filter=has_wikidata:false
filter="has_wikidata:false" # @param "has_wikidata:false" {type: "string"}

response = concepts_api.get_concepts(
	filter=filter
)

df = pd.DataFrame(response.results)
display(df)

In [ ]:
numeric_df = df[['id', 'display_name'] +
	[col for col in df.columns if df[col].dtype in ['int64', 'float64'] and col != 'relevance_score']]
display(numeric_df)

try:
	llm = OpenAI(api_token = openapi_token)
	sdf = SmartDataframe(numeric_df, config = { "llm": llm })
	sdf.chat("Plot a chart of this data")
except:
	if not openapi_token:
		print("Error: openapi_token not set")
	else:
		print("Error when creating SmartDataframe")